In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np

pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 500)
pd.set_option("display.width", 1000)
pd.set_option("display.max_columns", None)
pd.set_option("max_colwidth", 800)
pd.set_option("display.max_colwidth", None)

import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)

In [3]:
artists = ["Sanah"]

artists = [x.replace(" ", "_").lower() for x in artists]
artists

['sanah']

In [14]:
def song_scraper(artists: list):
    dfs = []
    df_text = pd.DataFrame(data={"Original": [], "Translated": []})

    base_url = "https://www.tekstowo.pl"

    for i in artists:
        artist_url = (
            f"https://www.tekstowo.pl/piosenki_artysty,{i},alfabetycznie,strona,1.html"
        )

        artist_page = requests.get(artist_url)
        soup = BeautifulSoup(artist_page.text, "lxml")

        max_num_page = soup.find_all("a", class_="page-link")[-2].text
        max_num_page = int(max_num_page)

        for num_page in range(1, max_num_page + 1):
            boxes = soup.find_all("div", class_="flex-group")
            songs_urls = {"Url": []}

            for box in boxes:
                try:
                    url = box.find("a", class_="title").get("href")
                    song_url = base_url + url

                    songs_urls["Url"].append(song_url)
                except:
                    pass

            df = pd.DataFrame(songs_urls)
            dfs.append(df)

            next_number_page = num_page + 1
            next_page = f"https://www.tekstowo.pl/piosenki_artysty,{i},alfabetycznie,strona,{next_number_page}.html"

            page = requests.get(next_page)
            soup = BeautifulSoup(page.text, "lxml")

    df_songs = pd.concat(dfs, ignore_index=True)

    for _, row in df_songs.iterrows():
        try:
            song_page = requests.get(row[0])

            soup = BeautifulSoup(song_page.text, "lxml")

            original = soup.find_all("div", class_="inner-text")[0].text
            translated = soup.find_all("div", class_="inner-text")[1].text
            translated = np.nan if translated == "" else translated

            if re.search(r"[ąćęłńóśźżĄĆĘŁŃÓŚŹŻ]", original):
                df_text = pd.concat(
                    [df_text, pd.DataFrame({"Original": [original], "Translated": [translated]})],
                    ignore_index=True
                )

            else:
                df_text = pd.concat(
                    [df_text, pd.DataFrame({"Original": [translated], "Translated": [original]})],
                    ignore_index=True
                )
        except:
            pass 

    return df_text


df = song_scraper(artists)

In [17]:
df.head()

,Original,Translated
0,"Da Bóg kiedyś zasiąść w Polsce wolnej\r\nOd żyta złotej, od lasów szumiącej\r\nDa Bóg, a przyjdzie dzień nieustający\r\nDla srebrnych pługów udręki mozolnej\r\nJeszcze oddźwiękną kamienie na młoty\r\nI z twardym ziemią pogada lemieszem\r\nI z wszystkich jeszcze kamieni wykrzeszem\r\nIskier snop złoty\n\r\nWięc gdy wiosennym oglądam wieczorem\r\nW mgły otuloną zagonów szarzyznę\r\nKtoś w moim sercu wykuwa toporem\r\nMoją Ojczyznę\n\r\nWięc gdy wiosennym oglądam wieczorem\r\nW mgły otuloną zagonów szarzyznę\r\nKtoś w moim sercu wykuwa toporem\r\nMoją Ojczyznę","Let God someday take a seat in a free Poland\nFrom golden rye, from forests rustling\nGod will give, and the day will come without ceasing\nFor the silver plows of toil\nHammer stones will still sound\nAnd talks with the hard earth with a plowshare\nAnd from all the stones I will still make\nGolden sheaf of sparks\n\nSo when I watch spring evening\nGrayness wrapped in the mist\nSomeone in my heart is forging with an axe\nmy homeland\n\nSo when I watch spring evening\nGrayness wrapped in the mist\nSomeone in my heart is forging with an axe\nmy homeland"
1,"Da Bóg kiedyś zasiąść w Polsce wolnej\nOd żyta złotej, od lasów szumiącej\nDa Bóg, a przyjdzie dzień nieustający\nDla srebrnych pługów udręki mozolnej\n\nJeszcze oddźwiękną kamienie na młoty\nI z twardym ziemią pogada lemieszem\nI z wszystkich jeszcze kamieni wykrzeszem\nIskier snop złoty\n\nWięc gdy wiosennym oglądam wieczorem\nW mgły otuloną zagonów szarzyznę\nKtoś w moim sercu wykuwa toporem\nMoją Ojczyznę\n\nWięc gdy wiosennym oglądam wieczorem\nW mgły otuloną zagonów szarzyznę\nKtoś w moim sercu wykuwa toporem\nMoją Ojczyznę\n\n[instrumental break]\n\nWięc gdy wiosennym oglądam wieczorem\nW mgły otuloną zagonów szarzyznę\nKtoś w moim sercu wykuwa toporem\nMoją Ojczyznę\n\nWięc gdy wiosennym oglądam wieczorem\nW mgły otuloną zagonów szarzyznę\nKtoś w moim sercu wykuwa toporem\nMoją Ojczyznę","Let God someday take a seat in a free Poland\nFrom golden rye, from forests rustling\nGod will give, and the day will come without ceasing\nFor the silver plows of toil\n\nHammer stones will still sound\nAnd talks with the hard earth with a plowshare\nAnd from all the stones I will still make\nGolden sheaf of sparks\n\nSo when I watch spring evening\nGrayness wrapped in the mist\nSomeone in my heart is forging with an axe\nmy homeland\n\nSo when I watch spring evening\nGrayness wrapped in the mist\nSomeone in my heart is forging with an axe\nmy homeland\n\n[instrumental break]\n\nSo when I watch spring evening\nGrayness wrapped in the mist\nSomeone in my heart is forging with an axe\nmy homeland\n\nSo when I watch spring evening\nGrayness wrapped in the mist\nSomeone in my heart is forging with an axe\nmy homeland"
2,"Hejka tu Sanah\r\nLaska od szampana\r\nO to posądzana\r\nŻe go pije\r\nZ rana\r\nZ rana\n\r\nZeusa słuchałam\r\nGdy 18 lat miałam\r\nNominację od niego dostałam\r\nI tego\r\nChciałam\r\nChciałam\r\nChciałam\r\nChciałam\n\r\nA ja o sobie mówię królowa dram\r\nMych łez uzbierałam już cały dzban\r\nMoże na piątej płycie będzie ich brak\r\nLecz dziś spory zapas ich mam\n\r\nSzkoda, że\r\nCzas tak szybko płynie\r\nChwila, dwie\r\nA nutka ta przeminie\r\nMoże ktoś\r\nPrzy niej odpłynie\n\r\nPiosnka ta\r\nMoże nie najlepsza\r\nAle płynie\r\nProsto z mego serca\r\nNawet jeśli\r\nNie jest wybitna\n\r\nByły łezki, proszę pana\r\nMokry parkiet od szampana\r\nI koronki i buciki\r\nZnacie wszystkie moje triki\r\nUps\r\nZatem no cóż\r\nWrzucam na luz\r\nGdyby ktoś się pytał właśnie wyszła moja płyta","Hey, it's Sanah\nThe champagne shawty\nSuspected of\nDrinking it\nIn the morning\nIn the morning\n\nI listened to Zeus\nWhen I was 18\nGot a nomination from him\nAnd I\nWanted it\nWanted it\nWanted it\nWanted it\n\nI call myself the drama queen\nI've already collected a whole jug of my tears\nMaybe they'll be missing on the fifth album\nBut today I have a lot of them\n\nIt's a pity that\nTime goes by so fast

In [18]:
df.duplicated().sum()

0